# Problem 1: Internet Investigation

## Problem Statement

X has heard complaints regarding the speed of their WiFi network. The network consists of r routers, some of which are marked as entry points which are connected to the rest of the internet. Some pairs of routers are directly connected to each other via bidirectional wires. Each wire wi between two routers has a known length ℓi measured in a positive integer number of feet. 

The latency of a router in the network is proportional to the minimum feet of wire a signal from the router must pass through to reach an entry point. Assume the latency of every router is finite and there is at most 100r feet of wire in the entire network.

**Given:** A schematic of the network depicting all routers and the lengths of all wires

**Find:** An O(r)-time algorithm to determine the sum total latency, summed over all routers in the network.

---

## Solution Explanation

### Key Insight
Transform the weighted shortest path problem into an unweighted shortest path problem, then use BFS (Breadth-First Search) to solve it efficiently.

### Visual Example

Let's work through a concrete example:

**Original Network:**
```
🌐 Entry Point A ——————5 feet—————— Router B ——3 feet—— Router C
                                    |
                                 2 feet
                                    |
                        🌐 Entry Point D
```

**Goal:** Find latency for each router (minimum distance to any entry point)

---

## Algorithm Steps

### Step 1: Transform Weighted Edges to Unweighted Chains

**Key Idea:** Replace each wire of length ℓ with a chain of ℓ unit-length edges.

**Before (weighted graph):**
```
A ——————5 feet—————— B ——3 feet—— C
        |
     2 feet
        |
        D
```

**After (unweighted graph):**
```
A —— ○ —— ○ —— ○ —— ○ —— B —— ○ —— ○ —— C
                            |
                            ○
                            |
                            D
```

Each `○` represents an intermediate node, and each `——` represents a unit-length edge.

### Step 2: Add Super Node

Add a special "super node" S connected to all entry points with unit-length edges:

```
                    S (SUPER NODE)
                   / \
                  1   1
                 /     \
               🌐A —○—○—○—○— B —○—○— C
                            |
                            ○
                            |
                           🌐D
```

### Step 3: Run BFS from Super Node

Perform Breadth-First Search starting from the super node S:

**BFS Exploration by Levels:**

```
Level 0: [S]
         ↓
Level 1: [A, D]  ← Entry points (distance 1 from S)
         ↓
Level 2: [○, ○]  ← First intermediate nodes from A and from D→B path
         ↓
Level 3: [○, B]  ← More intermediate nodes, and B is reached!
         ↓
Level 4: [○, ○]  ← Continue exploring
         ↓
Level 5: [○, ○]  ← More intermediate nodes
         ↓
Level 6: [C]     ← C is finally reached!
```

**Distance Results:**
- Distance from S to A = 1
- Distance from S to D = 1  
- Distance from S to B = 3
- Distance from S to C = 6

### Step 4: Calculate Actual Latencies

Since we added the super node, each distance is 1 more than the actual latency:

- **Entry Point A:** Latency = Distance - 1 = 1 - 1 = **0 feet**
- **Entry Point D:** Latency = Distance - 1 = 1 - 1 = **0 feet**
- **Router B:** Latency = Distance - 1 = 3 - 1 = **2 feet**
- **Router C:** Latency = Distance - 1 = 6 - 1 = **5 feet**

**Total Latency = 0 + 0 + 2 + 5 = 7 feet**

---

## Algorithm Pseudocode

```
Algorithm: ComputeTotalLatency(routers, wires, entry_points)

1. // Step 1: Build unweighted graph
   G = empty graph
   
   For each router r:
       Add vertex r to G
   
   For each wire (u, v, length):
       Create chain: u —○—○—...—○— v (length edges, length-1 intermediate nodes)
       Add all edges and intermediate nodes to G
   
2. // Step 2: Add super node
   Add super node S to G
   For each entry_point e:
       Add edge (S, e) to G
   
3. // Step 3: Run BFS
   distances = BFS(G, start_node=S)
   
4. // Step 4: Calculate result
   total_latency = 0
   For each router r:
       latency = distances[r] - 1
       total_latency += latency
   
   Return total_latency
```

---

## Time Complexity Analysis

### Graph Size Analysis

**Vertices:**
- Original routers: `r` vertices
- Intermediate vertices from wires: At most `100r` vertices
  (since total wire length ≤ 100r feet, and each foot creates 1 intermediate vertex)
- Super node: `1` vertex
- **Total vertices: r + 100r + 1 = O(r)**

**Edges:**
- Wire edges: At most `100r` edges (total wire length)
- Super node edges: `r` edges (one to each entry point)
- **Total edges: 100r + r = O(r)**

### Time Complexity

1. **Graph Construction:** O(r)
   - Processing each wire takes time proportional to its length
   - Total wire length ≤ 100r, so O(r) time

2. **BFS Execution:** O(V + E) = O(r + r) = O(r)
   - V = O(r) vertices, E = O(r) edges

3. **Result Calculation:** O(r)
   - Sum over r routers

**Total Time Complexity: O(r)**

---

## Why This Algorithm Works

### Correctness
1. **Transformation Preserves Distances:** Converting each ℓ-foot wire into ℓ unit edges preserves shortest path distances
2. **Super Node Optimization:** Instead of running BFS from each entry point separately, the super node lets us find shortest paths from ALL entry points in one BFS run
3. **BFS Finds Shortest Paths:** In unweighted graphs, BFS correctly computes shortest path distances

### Efficiency
- The key insight is that even though we add intermediate nodes, the total graph size remains O(r) due to the constraint that total wire length ≤ 100r
- This allows us to use the linear-time BFS algorithm instead of more expensive shortest-path algorithms like Dijkstra's

---

## Alternative Approaches (Less Efficient)

**Naive Approach:** Run Dijkstra's algorithm from each entry point
- Time Complexity: O(r × E log V) where E, V could be large
- Much slower than O(r)

**Multiple BFS Approach:** Run BFS from each entry point in the transformed graph
- Time Complexity: O(entry_points × r)
- Could be worse than O(r) if many entry points exist

The super node trick is crucial for achieving O(r) time complexity!